### Importación de archivos y librerías

In [75]:
! pip install sqlalchemy==1.4.46
! pip install pandasql==0.7.3
! pip install xgboost


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [76]:
import sys
sys.path.append("../")
from sales import data_holder, data_info, common, feature_engineering
import pandas as pd

### Generación de dataframes

In [77]:
data = pd.read_csv('../data/data sales csv/sales_data.csv', delimiter=',')
data_holder.set_data(data)
data_info.preview()

Data shape:  (1162971, 29)


,FechaFactura,mes,business_unit,team,category,class,gender,tipo_producto,sku,cc,...,CURRENCYCODE,AfectoFinal,CantVendida,Promedio de Mg,product_name,familia(collection),Rut_Cliente,Año,Semana,Season_fecha
0,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$2.479,1,100%,NaN,NaN,10530209-6,2023,6,SP22
1,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$0,1,NaN,NaN,NaN,13620253-7,2023,6,SP22
2,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$0,1,NaN,NaN,NaN,15170047-0,2023,6,SP22
3,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$0,1,NaN,NaN,NaN,15327358-8,2023,6,SP22
4,08-02-2023,2,NaN,NaN,NaN,NaN,NaN,NaN,DESPACHO,NaN,...,CLP,$3.319,1,100%,NaN,NaN,16711666-3,2023,6,SP22


In [78]:
df_warehouses = pd.read_csv('../data/data warehouses/warehouses.csv', delimiter=';')
df_warehouses.head()

,warehouse,is_metropolitan_zone,is_inside_mall,Unnamed: 3
0,LASCONDES,1,1,NaN
1,COSTANERA,1,1,NaN
2,LADEHESA,1,1,NaN
3,MALLSPORT,1,1,NaN
4,PTOVARAS,0,0,NaN


In [79]:
df_sale_campaigns = pd.read_csv('../data/data sale campaigns/sale_campaigns.csv', delimiter=';', encoding='latin-1')
df_sale_campaigns = df_sale_campaigns.rename(columns={
    'Fecha Inicio': 'start_date',
    'Fecha Fin': 'end_date',
    'Descripción': 'campaign_name'
})
df_sale_campaigns.head()

,start_date,end_date,campaign_name
0,01-03-2019,17-03-2019,SALE
1,01-05-2019,12-05-2019,DIA DE LA MADRE
2,05-06-2019,16-06-2019,DIA DEL PADRE
3,30-08-2019,23-09-2019,SALE
4,21-02-2020,15-03-2020,SALE


### Data preprocessing

In [80]:
data_info.duplicates()
data_holder.delete_duplicates()
data_info.duplicates()

Duplicates in Data set: 2602,(0.22%)
Duplicates in Data set: 0,(0.0%)


We remove all the items that refer to the shipping charge

In [81]:
data_holder.remove_rows_by_values(['sku'], ['DESPACHO'])
data_holder.remove_rows_by_values(['sku'], ['ENVIO'])
data_info.preview()

Data shape:  (1075322, 29)


,FechaFactura,mes,business_unit,team,category,class,gender,tipo_producto,sku,cc,...,CURRENCYCODE,AfectoFinal,CantVendida,Promedio de Mg,product_name,familia(collection),Rut_Cliente,Año,Semana,Season_fecha
16,08-02-2023,2,LIFE OUTDOORS,M S SPORTSWEAR,TOPS,S/S TOPS,MENS,CAMISAS,42245CYPIM,42245CYPI,...,CLP,$32.773,1,68%,M S DAILY SHIRT,OTROS,19061589-8,2023,6,SP22
17,08-02-2023,2,LIFE OUTDOORS,W S SPORTSWEAR,BOTTOMS,PANTS,WOMENS,PANTALONES,56566WWIBS,56566WWIB,...,CLP,$41.176,1,69%,W S GARDEN ISLAND PANTS,OTROS,9194419-7,2023,6,SP22
18,08-02-2023,2,MOUNTAINS,ALPINE,OUTERWEAR,OUTERWEAR JACKETS,MENS,IMPERMEABLES,85240BLKM,85240BLK,...,CLP,$104.202,1,64%,M S TORRENTSHELL 3L JKT,TORRENTSHELL,16923253-9,2023,6,SP22
19,08-02-2023,2,MOUNTAINS,ALPINE,OUTERWEAR,OUTERWEAR JACKETS,MENS,PARKAS,84701CNYL,84701CNY,...,CLP,$60.252,1,3%,M S DOWN SWEATER HOODY,DOWN SWEATER,19133957-6,2023,6,SP22
20,08-02-2023,2,MOUNTAINS,ALPINE,TOPS,MIDLAYER JACKETS,MENS,CHAQUETAS,83580BLKL,83580BLK,...,CLP,$91.597,1,65%,M S R1 TECHFACE JKT,TECHFACE,16711666-3,2023,6,SP22


In [82]:
df = data_holder.get_data()
print(df[df['CantVendida'] <= 0].size)
df = df[df['CantVendida'] > 0]
data_holder.set_data(df)

2674264


In [83]:
data_holder.remove_columns_by_features([
       'FechaFactura', 'sku', 'cc', 'itemnumber', 'TiendaDim',
       'CostoTotal', 'AfectoFinal', 'CantVendida'
])
data_info.preview()

Data shape:  (983106, 8)


,FechaFactura,sku,cc,itemnumber,TiendaDim,CostoTotal,AfectoFinal,CantVendida
16,08-02-2023,42245CYPIM,42245CYPI,42245,Ecommerce,$10.496,$32.773,1
17,08-02-2023,56566WWIBS,56566WWIB,56566,Ecommerce,$12.740,$41.176,1
18,08-02-2023,85240BLKM,85240BLK,85240,Ecommerce,$37.794,$104.202,1
19,08-02-2023,84701CNYL,84701CNY,84701,Ecommerce,$58.591,$60.252,1
20,08-02-2023,83580BLKL,83580BLK,83580,Ecommerce,$32.175,$91.597,1


In [84]:
data_holder.rename_columns({
                    'FechaFactura': 'date',
                    'TiendaDim': 'warehouse',
                    'CostoTotal': 'cost',
                    'AfectoFinal': 'sale_amount', 
                    'CantVendida': 'quantity',
                    })
data_info.preview()

Data shape:  (983106, 8)


,date,sku,cc,itemnumber,warehouse,cost,sale_amount,quantity
16,08-02-2023,42245CYPIM,42245CYPI,42245,Ecommerce,$10.496,$32.773,1
17,08-02-2023,56566WWIBS,56566WWIB,56566,Ecommerce,$12.740,$41.176,1
18,08-02-2023,85240BLKM,85240BLK,85240,Ecommerce,$37.794,$104.202,1
19,08-02-2023,84701CNYL,84701CNY,84701,Ecommerce,$58.591,$60.252,1
20,08-02-2023,83580BLKL,83580BLK,83580,Ecommerce,$32.175,$91.597,1


In [85]:
data_info.get_types()

Data types: 

date           object
sku            object
cc             object
itemnumber     object
warehouse      object
cost           object
sale_amount    object
quantity        int64
dtype: object


In [86]:
df = data_holder.get_data()   
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
df['sale_amount'] = df['sale_amount'].str.replace('[$.]', '').astype(float).astype(int)
df['sale_amount'] = df['sale_amount'].astype(int)
df['cost'] = df['cost'].str.replace('[$.]', '').astype(float).astype(int)
df['cost'] = df['cost'].astype(int)
df['quantity'] = df['quantity'].astype(int)

data_holder.set_data(df)
df.head()

C:\Users\Enrique Urrutia\AppData\Local\Temp\ipykernel_17368\1163990109.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['sale_amount'] = df['sale_amount'].str.replace('[$.]', '').astype(float).astype(int)
C:\Users\Enrique Urrutia\AppData\Local\Temp\ipykernel_17368\1163990109.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['cost'] = df['cost'].str.replace('[$.]', '').astype(float).astype(int)


,date,sku,cc,itemnumber,warehouse,cost,sale_amount,quantity
16,2023-02-08,42245CYPIM,42245CYPI,42245,Ecommerce,10496,32773,1
17,2023-02-08,56566WWIBS,56566WWIB,56566,Ecommerce,12740,41176,1
18,2023-02-08,85240BLKM,85240BLK,85240,Ecommerce,37794,104202,1
19,2023-02-08,84701CNYL,84701CNY,84701,Ecommerce,58591,60252,1
20,2023-02-08,83580BLKL,83580BLK,83580,Ecommerce,32175,91597,1


Delete rows that refer to *Pop Up* warehouse 

In [87]:
df = data_holder.get_data()
print(df[df['warehouse'] == 'Pop UP'].shape)
df = df[df['warehouse'] != 'Pop UP']
data_holder.set_data(df)

(39696, 8)


Standardize *warehouses* names

In [88]:
data_holder.get_data()['warehouse'].unique()

array(['Ecommerce', 'Concepción', 'Puerto Varas', 'Temuco', 'Alerce',
       'Pucón', 'Coyhaique', 'Costanera', 'La Dehesa', 'Mall Sport',
       'Osorno', 'BuenaVentura', 'Wholesale', 'Alto Las Condes',
       'El Golf'], dtype=object)

In [89]:
warehouse_names = {
    'Alto Las Condes': 'LASCONDES', 'Mall Sport': 'MALLSPORT',
    'La Dehesa': 'LADEHESA', 'Pucón': 'PUCON', 'Temuco': 'TEMUCO',
    'Concepción': 'CONCEPCION', 'Coyhaique': 'COYHAIQUE',
    'Puerto Varas': 'PTOVARAS', 'BuenaVentura': 'BUENAVENTURA',
    'Wholesale': ' WHOLESALE', 'Ecommerce': 'ECOMMERCE',
    'Costanera': 'COSTANERA', 'El Golf': 'ELGOLF',
    'Alerce': 'ALERCE', 'Osorno': 'OSORNO'
}
df['warehouse'] = df['warehouse'].apply(lambda x: warehouse_names[x])

data_holder.set_data(df)
df['warehouse'].unique()

array(['ECOMMERCE', 'CONCEPCION', 'PTOVARAS', 'TEMUCO', 'ALERCE', 'PUCON',
       'COYHAIQUE', 'COSTANERA', 'LADEHESA', 'MALLSPORT', 'OSORNO',
       'BUENAVENTURA', ' WHOLESALE', 'LASCONDES', 'ELGOLF'], dtype=object)

#### Codificate to unit price and group data for date-sku-warehouse

In [90]:
data_holder.set_data(
    feature_engineering.add_unit_price_information(data_holder.get_data())
)
data_info.preview()

Data shape:  (943410, 9)


,date,sku,cc,itemnumber,warehouse,cost,sale_amount,quantity,unit_price
0,2023-02-08,42245CYPIM,42245CYPI,42245,ECOMMERCE,10496,32773,1,32773
1,2023-02-08,56566WWIBS,56566WWIB,56566,ECOMMERCE,12740,41176,1,41176
2,2023-02-08,85240BLKM,85240BLK,85240,ECOMMERCE,37794,104202,1,104202
3,2023-02-08,84701CNYL,84701CNY,84701,ECOMMERCE,58591,60252,1,60252
4,2023-02-08,83580BLKL,83580BLK,83580,ECOMMERCE,32175,91597,1,91597


In [74]:
data_holder.group_data(
        ['date', 'sku', 'cc', 'itemnumber', 'warehouse'],
        {'cost' : 'mean',
         'unit_price': 'mean',
         'sale_amount' : 'max',
         'quantity' : 'sum'})
data_info.preview()

Data shape:  (876141, 8)


,date,sku,cc,warehouse,cost,unit_price,sale_amount,quantity
0,2018-08-02,27810NUVGM,27810NUVG,LASCONDES,0.0,8739.0,8739,1
1,2018-08-02,28322FYBBL,28322FYBB,LASCONDES,0.0,13782.0,13782,1
2,2018-08-02,44422BLKXL,44422BLK,LASCONDES,0.0,8739.0,8739,1
3,2018-08-03,12010ZOOALL,12010ZOO,MALLSPORT,72588.0,50084.0,50084,1
4,2018-08-03,12011ZOOALL,12011ZOO,LADEHESA,81820.0,56807.0,56807,1


### Feature engineering

#### Add *Date* relate information

In [18]:
print("#### Adding Date related info")
df = data_holder.get_data()

df = feature_engineering.add_weekday_information(df)
print("Ok feature 1")
df = feature_engineering.add_month_information(df)
print("Ok feature 2")
df = feature_engineering.add_year_information(df)
print("Ok feature 3")
df = feature_engineering.add_offer_day_information(df, df_sale_campaigns)
print("Ok feature 4")

data_holder.set_data(df)
data_info.preview()

Data shape:  (2647992, 11)


,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,year,is_offer_day
0,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1
1,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1
2,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1
3,2018-08-02,28322FYBBL,LASCONDES,0.0,13782.0,13782,1,4,8,2018,1
4,2018-08-02,28322FYBBL,LASCONDES,0.0,13782.0,13782,1,4,8,2018,1


#### Add *Warehouse* relate information

In [19]:
warehouse_dummies = pd.get_dummies(df['warehouse'], prefix='warehouse')
warehouse_dummies

,warehouse_ WHOLESALE,warehouse_ALERCE,warehouse_BUENAVENTURA,warehouse_CONCEPCION,warehouse_COSTANERA,warehouse_COYHAIQUE,warehouse_ECOMMERCE,warehouse_ELGOLF,warehouse_LADEHESA,warehouse_LASCONDES,warehouse_MALLSPORT,warehouse_OSORNO,warehouse_PTOVARAS,warehouse_PUCON,warehouse_TEMUCO
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647987,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2647988,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2647989,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2647990,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [20]:
print("#### Adding Warehouse related info")
df = data_holder.get_data()

df = feature_engineering.add_warehouse_is_metropolitan_zone(df, df_warehouses)
print("Ok feature 1")
df = feature_engineering.add_warehouse_is_inside_mall(df, df_warehouses)
print("Ok feature 2")

data_holder.set_data(df)
data_info.preview()

Data shape:  (2647992, 13)


,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,year,is_offer_day,is_metropolitan_zone,is_inside_mall
0,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1,1,1
1,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1,1,1
2,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1,1,1
3,2018-08-02,28322FYBBL,LASCONDES,0.0,13782.0,13782,1,4,8,2018,1,1,1
4,2018-08-02,28322FYBBL,LASCONDES,0.0,13782.0,13782,1,4,8,2018,1,1,1


#### Add *SKU - Date*  related info

In [21]:
print("#### Adding Sku-date related info")
df = data_holder.get_data()

df = feature_engineering.add_sku_cumulative_sales_in_the_week(df)
print("Ok feature 1")
df = feature_engineering.add_sku_cumulative_sales_in_the_month(df)
print("Ok feature 2")
df = feature_engineering.add_sku_cumulative_sales_in_the_year(df)
print("Ok feature 3")
df = feature_engineering.add_sku_historic_sales(df)
print("Ok feature 4")
df = feature_engineering.add_sku_historic_sales_same_day_of_the_week(df)
print("Ok feature 5")
df = feature_engineering.add_sku_historic_sales_same_month(df)
print("Ok feature 6")

data_holder.set_data(df)
data_info.preview()

Data shape:  (882664, 19)


,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,year,is_offer_day,is_metropolitan_zone,is_inside_mall,sku_cumulative_sales_in_the_week,sku_cumulative_sales_in_the_month,sku_cumulative_sales_in_the_year,sku_historic_sales,sku_historic_sales_same_day_of_the_week,sku_historic_sales_same_month
0,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1,1,1,0,0,0,0,0,0
1,2018-08-02,28322FYBBL,LASCONDES,0.0,13782.0,13782,1,4,8,2018,1,1,1,0,0,0,0,0,0
2,2018-08-02,44422BLKXL,LASCONDES,0.0,8739.0,8739,1,4,8,2018,1,1,1,0,0,0,0,0,0
3,2018-08-03,12010ZOOALL,MALLSPORT,72588.0,50084.0,50084,1,5,8,2018,1,1,1,0,0,0,0,0,0
4,2018-08-03,12011ZOOALL,LADEHESA,81820.0,56807.0,56807,1,5,8,2018,1,1,1,0,0,0,0,0,0


#### Add *SKU-Warehouse-Date* related info

In [22]:
print("#### Adding Sku-warehouse-date related info")
df = data_holder.get_data()

df = feature_engineering.add_sku_warehouse_last_xdays_sales(df, 7)
print("Ok feature 1")
df = feature_engineering.add_sku_warehouse_last_xdays_mean_sales(df, 7)
print("Ok feature 2")
df = feature_engineering.add_sku_warehouse_last_xdays_sales(df, 14)
print("Ok feature 3")
df = feature_engineering.add_sku_warehouse_last_xdays_mean_sales(df, 14)
print("Ok feature 4")
df = feature_engineering.add_sku_warehouse_last_xdays_sales(df, 30)
print("Ok feature 5")
df = feature_engineering.add_sku_warehouse_last_xdays_mean_sales(df, 30)
print("Ok feature 6")

df = feature_engineering.add_sku_warehouse_historic_sales(df)
print("Ok feature 7")
df = feature_engineering.add_sku_warehouse_historic_sales_same_day_of_the_week(df)
print("Ok feature 8")
df = feature_engineering.add_sku_warehouse_historic_sales_same_month(df)
print("Ok feature 9")

df = feature_engineering.add_sku_warehouse_cumulative_sales_in_the_week(df)
print("Ok feature 10")
df = feature_engineering.add_sku_warehouse_cumulative_sales_in_the_month(df)
print("Ok feature 11")
df = feature_engineering.add_sku_warehouse_cumulative_sales_in_the_year(df)
print("Ok feature 12")

df = feature_engineering.add_y_sku_warehouse_next_xdays_sales(df, 7)
print("Ok feature 13")
df = feature_engineering.add_y_sku_warehouse_next_xdays_sales(df, 14)
print("Ok feature 14")
df = feature_engineering.add_y_sku_warehouse_next_xdays_sales(df, 30)
print("Ok feature 15")


data_holder.set_data(df)
data_info.preview()

Ok feature 1
Ok feature 2
Ok feature 3
Ok feature 4
Ok feature 5
Ok feature 6
Ok feature 7
Ok feature 8
Ok feature 9
Ok feature 10
Ok feature 11
Ok feature 12
Data shape:  (882664, 31)


,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,year,...,sku_warehouse_last_14days_sales,sku_warehouse_last_14days_mean_sales,sku_warehouse_last_30days_sales,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year
0,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,2018,...,0,0.0,0,0.0,0,0,0,0,0,0
1,2018-08-02,28322FYBBL,LASCONDES,0.0,13782.0,13782,1,4,8,2018,...,0,0.0,0,0.0,0,0,0,0,0,0
2,2018-08-02,44422BLKXL,LASCONDES,0.0,8739.0,8739,1,4,8,2018,...,0,0.0,0,0.0,0,0,0,0,0,0
3,2018-08-03,12010ZOOALL,MALLSPORT,72588.0,50084.0,50084,1,5,8,2018,...,0,0.0,0,0.0,0,0,0,0,0,0
4,2018-08-03,12011ZOOALL,LADEHESA,81820.0,56807.0,56807,1,5,8,2018,...,0,0.0,0,0.0,0,0,0,0,0,0


In [31]:
data_holder.get_data().to_csv('../data/temp/processed_data.csv')

### Training the model

Import libraries

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
from sales import data_holder, data_info, common, feature_engineering
from xgboost import XGBRegressor

Read data produced with feature engineering

In [6]:
data = pd.read_csv('../data/temp/processed_data.csv', delimiter=',')
data_holder.set_data(data)
data_info.preview()

Data shape:  (882664, 32)


,Unnamed: 0,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,...,sku_warehouse_last_14days_sales,sku_warehouse_last_14days_mean_sales,sku_warehouse_last_30days_sales,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year
0,0,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,...,0,0.0,0,0.0,0,0,0,0,0,0
1,1,2018-08-02,28322FYBBL,LASCONDES,0.0,13782.0,13782,1,4,8,...,0,0.0,0,0.0,0,0,0,0,0,0
2,2,2018-08-02,44422BLKXL,LASCONDES,0.0,8739.0,8739,1,4,8,...,0,0.0,0,0.0,0,0,0,0,0,0
3,3,2018-08-03,12010ZOOALL,MALLSPORT,72588.0,50084.0,50084,1,5,8,...,0,0.0,0,0.0,0,0,0,0,0,0
4,4,2018-08-03,12011ZOOALL,LADEHESA,81820.0,56807.0,56807,1,5,8,...,0,0.0,0,0.0,0,0,0,0,0,0


In [9]:
df = data_holder.get_data()
df.tail()

,Unnamed: 0,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,...,sku_warehouse_last_14days_sales,sku_warehouse_last_14days_mean_sales,sku_warehouse_last_30days_sales,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year
882659,882659,2023-02-08,59179FRBKALL,ECOMMERCE,4463.0,15966.0,15966,1,3,2,...,1,1.0,1,1.00,23,1,0,1,1,9
882660,882660,2023-02-08,83580BLKL,ECOMMERCE,32175.0,91597.0,91597,1,3,2,...,1,1.0,1,1.00,25,3,2,0,1,2
882661,882661,2023-02-08,84701CNYL,ECOMMERCE,58591.0,60252.0,60252,1,3,2,...,0,1.0,0,1.00,69,11,0,0,0,4
882662,882662,2023-02-08,85240BLKM,ECOMMERCE,37794.0,104202.0,104202,1,3,2,...,2,1.2,2,1.14,272,44,3,1,2,11
882663,882663,2023-02-08,BK825000ALL,ECOMMERCE,10198.0,15966.0,15966,1,3,2,...,2,1.0,2,1.38,425,65,55,0,2,11


In [10]:
df.sort_values('date').head()

,Unnamed: 0,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,...,sku_warehouse_last_14days_sales,sku_warehouse_last_14days_mean_sales,sku_warehouse_last_30days_sales,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year
0,0,2018-08-02,27810NUVGM,LASCONDES,0.0,8739.0,8739,1,4,8,...,0,0.0,0,0.0,0,0,0,0,0,0
1,1,2018-08-02,28322FYBBL,LASCONDES,0.0,13782.0,13782,1,4,8,...,0,0.0,0,0.0,0,0,0,0,0,0
2,2,2018-08-02,44422BLKXL,LASCONDES,0.0,8739.0,8739,1,4,8,...,0,0.0,0,0.0,0,0,0,0,0,0
3,3,2018-08-03,12010ZOOALL,MALLSPORT,72588.0,50084.0,50084,1,5,8,...,0,0.0,0,0.0,0,0,0,0,0,0
4,4,2018-08-03,12011ZOOALL,LADEHESA,81820.0,56807.0,56807,1,5,8,...,0,0.0,0,0.0,0,0,0,0,0,0


In [11]:
df.tail()

,Unnamed: 0,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,...,sku_warehouse_last_14days_sales,sku_warehouse_last_14days_mean_sales,sku_warehouse_last_30days_sales,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year
882659,882659,2023-02-08,59179FRBKALL,ECOMMERCE,4463.0,15966.0,15966,1,3,2,...,1,1.0,1,1.00,23,1,0,1,1,9
882660,882660,2023-02-08,83580BLKL,ECOMMERCE,32175.0,91597.0,91597,1,3,2,...,1,1.0,1,1.00,25,3,2,0,1,2
882661,882661,2023-02-08,84701CNYL,ECOMMERCE,58591.0,60252.0,60252,1,3,2,...,0,1.0,0,1.00,69,11,0,0,0,4
882662,882662,2023-02-08,85240BLKM,ECOMMERCE,37794.0,104202.0,104202,1,3,2,...,2,1.2,2,1.14,272,44,3,1,2,11
882663,882663,2023-02-08,BK825000ALL,ECOMMERCE,10198.0,15966.0,15966,1,3,2,...,2,1.0,2,1.38,425,65,55,0,2,11


Generate train and test datasets

In [12]:
cutoff_date = '2023-01-01'
train_data = df[df['date'] < cutoff_date]
test_data = df[df['date'] >= cutoff_date]

In [13]:
train_data.tail()

,Unnamed: 0,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,...,sku_warehouse_last_14days_sales,sku_warehouse_last_14days_mean_sales,sku_warehouse_last_30days_sales,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year
855075,855075,2022-12-31,84212MEORL,ECOMMERCE,42688.0,150420.0,150420,1,6,12,...,0,0.0,0,0.0,4,0,0,0,0,4
855076,855076,2022-12-31,85245CNYS,ECOMMERCE,35727.0,83193.0,83193,1,6,12,...,1,1.0,1,1.0,15,0,0,1,5,15
855077,855077,2022-12-31,W65050EDWHPS,ECOMMERCE,4693.0,26050.0,26050,1,6,12,...,0,0.0,0,1.0,2,0,0,0,2,2
855078,855078,2022-12-31,W65485ETATGS,ECOMMERCE,2881.0,20168.0,20168,1,6,12,...,0,0.0,0,0.0,0,0,0,0,0,0
855079,855079,2022-12-31,W68475EBLSTM,ECOMMERCE,2881.0,33613.0,33613,1,6,12,...,0,0.0,0,0.0,0,0,0,0,0,0


In [39]:
df[(df['sku'] == '85245CNYS') &
   (df['warehouse'] == 'ECOMMERCE') &
   (df['month'] == 12)]

,Unnamed: 0,date,sku,warehouse,cost,unit_price,sale_amount,quantity,weekday,month,...,sku_warehouse_last_14days_sales,sku_warehouse_last_14days_mean_sales,sku_warehouse_last_30days_sales,sku_warehouse_last_30days_mean_sales,sku_warehouse_historic_sales,sku_warehouse_historic_sales_same_day_of_the_week,sku_warehouse_historic_sales_same_month,sku_warehouse_cumulative_sales_in_the_week,sku_warehouse_cumulative_sales_in_the_month,sku_warehouse_cumulative_sales_in_the_year
838062,838062,2022-12-13,85245CNYS,ECOMMERCE,33672.0,83193.0,83193,1,2,12,...,0,0.0,0,0.0,10,3,0,0,0,10
838986,838986,2022-12-14,85245CNYS,ECOMMERCE,33672.0,83193.0,83193,1,3,12,...,1,1.0,1,1.0,11,1,0,1,1,11
840385,840385,2022-12-15,85245CNYS,ECOMMERCE,33672.0,83193.0,83193,1,4,12,...,2,1.0,2,1.0,12,3,0,2,2,12
845440,845440,2022-12-19,85245CNYS,ECOMMERCE,33672.0,83193.0,83193,1,1,12,...,3,1.0,3,1.0,13,1,0,0,3,13
854964,854964,2022-12-29,85245CNYS,ECOMMERCE,35720.0,83193.0,83193,1,4,12,...,0,1.0,0,1.0,14,4,0,0,4,14
855076,855076,2022-12-31,85245CNYS,ECOMMERCE,35727.0,83193.0,83193,1,6,12,...,1,1.0,1,1.0,15,0,0,1,5,15


In [14]:
df.columns

Index(['Unnamed: 0', 'date', 'sku', 'warehouse', 'cost', 'unit_price',
       'sale_amount', 'quantity', 'weekday', 'month', 'year', 'is_offer_day',
       'is_metropolitan_zone', 'is_inside_mall',
       'sku_cumulative_sales_in_the_week', 'sku_cumulative_sales_in_the_month',
       'sku_cumulative_sales_in_the_year', 'sku_historic_sales',
       'sku_historic_sales_same_day_of_the_week',
       'sku_historic_sales_same_month', 'sku_warehouse_last_7days_sales',
       'sku_warehouse_last_7days_mean_sales',
       'sku_warehouse_last_14days_sales',
       'sku_warehouse_last_14days_mean_sales',
       'sku_warehouse_last_30days_sales',
       'sku_warehouse_last_30days_mean_sales', 'sku_warehouse_historic_sales',
       'sku_warehouse_historic_sales_same_day_of_the_week',
       'sku_warehouse_historic_sales_same_month',
       'sku_warehouse_cumulative_sales_in_the_week',
       'sku_warehouse_cumulative_sales_in_the_month',
       'sku_warehouse_cumulative_sales_in_the_year'],
      

In [17]:
# Split the data into X and y datasets
X_train = train_data.drop(['Unnamed: 0', 'date', 'sku', 'warehouse', 'cost', 'sale_amount', 'quantity'], axis=1)
y_train = train_data['quantity']
X_test = test_data.drop(['Unnamed: 0', 'date', 'sku', 'warehouse', 'cost', 'sale_amount', 'quantity'], axis=1)
y_test = test_data['quantity']

In [22]:
X_train.columns

Index(['unit_price', 'weekday', 'month', 'year', 'is_offer_day',
       'is_metropolitan_zone', 'is_inside_mall',
       'sku_cumulative_sales_in_the_week', 'sku_cumulative_sales_in_the_month',
       'sku_cumulative_sales_in_the_year', 'sku_historic_sales',
       'sku_historic_sales_same_day_of_the_week',
       'sku_historic_sales_same_month', 'sku_warehouse_last_7days_sales',
       'sku_warehouse_last_7days_mean_sales',
       'sku_warehouse_last_14days_sales',
       'sku_warehouse_last_14days_mean_sales',
       'sku_warehouse_last_30days_sales',
       'sku_warehouse_last_30days_mean_sales', 'sku_warehouse_historic_sales',
       'sku_warehouse_historic_sales_same_day_of_the_week',
       'sku_warehouse_historic_sales_same_month',
       'sku_warehouse_cumulative_sales_in_the_week',
       'sku_warehouse_cumulative_sales_in_the_month',
       'sku_warehouse_cumulative_sales_in_the_year'],
      dtype='object')

In [18]:
from xgboost import XGBRegressor
regression = XGBRegressor()
regression.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

Evaluate model performance

In [28]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
y_pred = regression.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)



print("R-squared [R2]: %.2f" % r2)
print("Mean Absolute Error [MAE]: %.2f" % mae)
print("Mean Squared Error [MSE]: %.2f" % mse)
print("Root Mean Squared Error [RMSE]: %.2f" % rmse)

R-squared [R2]: -0.42
Mean Absolute Error [MAE]: 0.25
Mean Squared Error [MSE]: 0.74
Root Mean Squared Error [RMSE]: 0.86


Cross Validation

In [35]:
X = df.drop(['Unnamed: 0', 'date', 'sku', 'warehouse', 'cost', 'sale_amount', 'quantity'], axis=1)
y = df['quantity']

In [36]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(regression, X, y, cv=5, scoring='neg_mean_squared_error')
mse_scores = -scores
rmse_scores = np.sqrt(mse_scores)
print("RMSE scores: ", rmse_scores)
print("Mean RMSE: %.2f" % rmse_scores.mean())

RMSE scores:  [1.6857245  1.55603311 1.05910882 1.62608793 2.73451161]
Mean RMSE: 1.73
